In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key='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'
    # other params...
)

response=llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://careers.nike.com/retail-associate-ii/job/R-63257")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Retail Associate II










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract=PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE).
    """
)

chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={'page_data': page_data})
print(res.content)

```json
[
  {
    "role": "Retail Associate II",
    "experience": "1-2 years",
    "skills": [
      "customer service",
      "retail experience",
      "communication",
      "teamwork",
      "basic retail math",
      "computer skills"
    ],
    "description": "Create a fun, energetic environment for our customers, consistently perform to operational and merchandising standards, partner with teammates on tasks, processes, merchandising and product flow opportunities."
  },
  {
    "role": "Converse Sales Associate",
    "experience": "1-2 years",
    "skills": [
      "customer service",
      "selling techniques",
      "merchandising",
      "product flow",
      "communication",
      "teamwork",
      "basic retail math",
      "computer skills"
    ],
    "description": "Drive the selling efforts and customer relationships in Converse stores through excelling in customer service and selling techniques, while supporting merchandising and product flow."
  }
]
```


In [7]:
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content) 

In [9]:
json_res

[{'role': 'Retail Associate II',
  'experience': '1-2 years',
  'skills': ['customer service',
   'retail experience',
   'communication',
   'teamwork',
   'basic retail math',
   'computer skills'],
  'description': 'Create a fun, energetic environment for our customers, consistently perform to operational and merchandising standards, partner with teammates on tasks, processes, merchandising and product flow opportunities.'},
 {'role': 'Converse Sales Associate',
  'experience': '1-2 years',
  'skills': ['customer service',
   'selling techniques',
   'merchandising',
   'product flow',
   'communication',
   'teamwork',
   'basic retail math',
   'computer skills'],
  'description': 'Drive the selling efforts and customer relationships in Converse stores through excelling in customer service and selling techniques, while supporting merchandising and product flow.'}]

In [10]:
type(json_res)

list

In [11]:
import pandas as pd

In [13]:
df=pd.read_csv('my_portfolio.csv')

In [14]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
import chromadb

client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row['Techstack'], metadatas={"links": row['Links']}, ids=[str(uuid.uuid4())])